<a href="https://colab.research.google.com/github/a798645635/CADD/blob/main/%E8%AF%BB%E5%8F%96%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive')


Mounted at /content/drive


In [ ]:
! pip uninstall -qq matplotlib
! pip install -qq matplotlib==1.5.3
! pip uninstall -qq matplotlib
! pip install -qq matplotlib==3.2.2

Proceed (Y/n)? y
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 MB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires matplotlib!=3.0.0,>=2.0.2, but you have matplotlib 1.5.3 which is incompatible.
seaborn 0.12.2 requires matplotlib!=3.6.1,>=3.1, but you have matplotlib 1.5.3 which is incompatible.
pycocotools 2.0.6 requires matplotlib>=2.1.0, but you have matplotlib 1.5.3 which is incompatible.
prophet 1.1.2 requires matplotlib>=2.0.0, but you have matplotlib 1.5.3 which is incompatible.
plotnine 0.10.1 requires matplotlib>=3.5.0, but you have matplotlib 1.5.3 which is incompatible.
phik 0.12.3 requires matplotlib>=2.2.3, but you have matplotlib 1.5.3 which is incompatible.
pandas-profiling 3.2.0 requires matplotlib>=3.2.0, but you have matplotlib 1.5.3 which is incom

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,LeaveOneOut,cross_val_score
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,make_scorer,matthews_corrcoef
from sklearn.metrics import confusion_matrix,roc_curve,auc
# import matplotlib.pyplot as plt
import matplotlib
import copy
import  pathlib as path
import joblib
import xgboost as xgb

In [ ]:
columns_backward=-3
train_path= r"/content/drive/MyDrive/data/MACCSKeysFingerprint_SOM_tr描述符缩减至70.csv"
test_path = r"/content/drive/MyDrive/data/MACCSKeysFingerprint_SOM_te描述符缩减至70.csv"

train_data=pd.read_csv(train_path)
test_data=pd.read_csv(test_path)

In [ ]:
# train_data = train_data.sample(frac=1,random_state=100)
# test_data = test_data.sample(frac=1,random_state=100)

test_x_index = test_data.columns[:columns_backward]
test_x=test_data[test_x_index]
test_y=test_data['y']

train_x_index = train_data.columns[:columns_backward]
train_x=train_data[train_x_index]
train_y=train_data['y']

#加载数据集
svc = joblib.load(r"/content/drive/MyDrive/data/Model D_1.model")
# dt =joblib.load(r"C:\Users\86150\Desktop\论文模型\Model D_2.model")
# rf=joblib.load(r"C:\Users\86150\Desktop\论文模型\Model C_3.model")
# xgboost=joblib.load(r"C:\Users\86150\Desktop\论文模型\Model D_4.model")


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
class Traditional_Model(object):
    
    def __init__(self,model,test_data,train_data):
    #初始化数据集
        self.best_clf=model
        self.test_data=test_data
        self.train_data=train_data



        self.test_x_index = self.test_data.columns[:columns_backward]
        self.test_x=self.test_data[test_x_index]
        self.test_y=self.test_data['y']
        self.train_x_index = self.train_data.columns[:columns_backward]
        self.train_x=self.train_data[train_x_index]
        self.train_y=self.train_data['y']

 
    def Draw_ROC(self,tifname,title):
        #定义Draw_ROC方法
        self.title =title
        self.tifname=tifname
        self.train_predict_pro = self.best_clf.predict_proba(self.train_x)
        self.test_predict_pro = self.best_clf.predict_proba(self.test_x)

        self.train_fpr,self.train_tpr,self.train_threshold = roc_curve(self.train_y,self.train_predict_pro[:,-1],drop_intermediate=False)
        self.test_fpr,self.test_tpr,self.test_threshold = roc_curve(self.test_y,self.test_predict_pro[:,-1],drop_intermediate=False)
        self.train_roc_auc = auc(self.train_fpr,self.train_tpr)
        self.test_roc_auc = auc(self.test_fpr,self.test_tpr)
        matplotlib.rcParams['font.sans-serif']=['simhei']
        matplotlib.rcParams['axes.unicode_minus']=False
        plt.figure(figsize=(12,9), dpi=800)
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.xticks(np.arange(0, 1.2, 0.2), fontproperties = 'Times New Roman', size = 18, weight='heavy')
        plt.yticks(np.arange(0, 1.2, 0.2), fontproperties = 'Times New Roman', size = 18, weight='heavy')
        #绘制训练集ROC曲线
        plt.plot(self.train_fpr,self.train_tpr,'r',label='Training set AUC = %0.2f'%self.train_roc_auc, linewidth=2)        
        plt.plot(self.test_fpr,self.test_tpr,'b',label='Test set AUC = %0.2f'%self.test_roc_auc, linewidth=2)
        #绘制测试集ROC曲线
        plt.legend(loc='lower right', prop={'family': 'Times New Roman', 'size': 24})
        #定义图例位置
        plt.plot([0,1],[0,1],'k--', linewidth=2)
        #绘制对角线
        plt.xlabel('False Positive Rate',fontdict={'family': 'Times New Roman', 'size': 30, 'weight': 'heavy'})
        #定义x轴标签
        plt.ylabel('True Positive Rate',fontdict={'family': 'Times New Roman', 'size': 30, 'weight': 'heavy'})
        #定义y轴标签
        plt.title(self.title,fontdict={'family': 'Times New Roman', 'size': 32, 'weight': 'heavy'})
        #定义标题
        plt.savefig(r"/content/drive/MyDrive/data/svc.tif",dpi=1200)
        plt.show()
        #显示图像

In [ ]:
%matplotlib inline
import matplotlib.style as style 
style.use('seaborn-deep')  

<ipython-input-7-6c8fff275497>:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  style.use('seaborn-deep')


In [ ]:
import matplotlib as mpl
import matplotlib.font_manager as fm
from matplotlib import font_manager as fm, pyplot as plt

!wget https://github.com/trishume/OpenTuringCompiler/blob/master/stdlib-sfml/fonts/Times%20New%20Roman.ttf


font_files = fm.findSystemFonts()






--2023-03-22 13:46:15--  https://github.com/trishume/OpenTuringCompiler/blob/master/stdlib-sfml/fonts/Times%20New%20Roman.ttf
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Times New Roman.ttf.4’

Times New Roman.ttf     [ <=>                ] 137.96K  --.-KB/s    in 0.08s   

2023-03-22 13:46:16 (1.73 MB/s) - ‘Times New Roman.ttf.4’ saved [141271]



In [ ]:
# Go through and add each to Matplotlib's font cache.
for font_file in font_files:
    fm.fontManager.addfont(font_file)

fm.fontManager.ttflist += fm.createFontList(['Times New Roman.ttf'])

AttributeError: ignored

In [ ]:
maccs_data = Traditional_Model(dt,test_data,train_data)
title="DT-MACCS-SOM"
maccs_data.Draw_ROC("DT-MACCS-SOM",title)
# maccs_data = Traditional_Model(svc,test_data,train_data)
# title="SVC-MACCS-SOM"
# maccs_data.Draw_ROC("SVC-MACCS-SOM",title)
maccs_data = Traditional_Model(xgboost,test_data,train_data)
title="XGB-MACCS-SOM"
maccs_data.Draw_ROC("XGB-MACCS-SOM",title)

In [ ]:
maccs_data = Traditional_Model(svc,test_data,train_data)
title="SVC-MACCS-SOM"
maccs_data.Draw_ROC("SVC-MACCS-SOM",title)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
